<a href="https://colab.research.google.com/github/chikaj/7316/blob/main/Real_Raster_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doing image stuff with Python like you do in ERDAS
***

## 0. There are a few preliminary steps first. We need data, for example. We will assume your data is on your Google Drive.
***
***
### 0.1 Mounting your Google Drive
***
* Run the following command to import __drive__ from __google.colab__.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 0.2 Now you need to be familiar with some Linux shell commands
***
* __ls__ (that's a lowercase L) will list the contents of your directory
* __pwd__ will list your present working directory
* __cd ```<directory>```__ will change directory to the ```<directory>``` you list
* __cd ..__ will change up one directory in the hierarchical tree of directories
* __mkdir <new_directory>__ will make a new directory in your _pwd_

##### Let's try these out now.

**What is your present working directory?**

In [ ]:
!ls
!pwd

**List the content of your /content/drive directory (aka folder).**

In [ ]:
ls


**Change into the MyDrive directory and list its contents.**

In [ ]:
cd drive/MyDrive

In [ ]:
ls

**Navigate to your data folder on your Google Drive**

In [ ]:
cd chelan/

**List the contents of this directory. Ah! Real raster image data. Landsat 8 to be precise.**

In [ ]:
ls

### 0.3 We now need to prepare Google Colab to use some of the packages/modules we need to do our work.
***

This page you are working on is a Jupyter Notebook. One way you can tell is that the file ends in an .ipynb extension. (ipynb is for IPython Notebook and Jupyter used to be called IPython.) Besides, this is what a Jupyter Notebook looks like...cells with text and code where each line can be executed separately from the other lines of code.

We are running Python in the Jupyter Notebook and Python uses a program called pip to install its packages. We'll do the same here. Let's install the rasterio package.

In [ ]:
!pip install rasterio

You can see that rasterio was installed...and so were other packages on which rasterio depends. We get it all using the !pip command.

## 1.0 Using Rasterio
***
***
### 1.1 Reading image data and associated attributes
***

In [ ]:
import rasterio as r

In [ ]:
rast = r.open('LC08_L1TP_045027_20210702_20210713_01_T1_B5.tiff')

In [ ]:
print(type(rast))

In [ ]:
print(dir(r.io.DatasetReader))

In [ ]:
print(f"The number of rows is {rast.height} and the number of columns is {rast.width}")

In [ ]:
print(f"The number of bands is {rast.count}")

In [ ]:
rast.shape

In [ ]:
rast.crs

In [ ]:
from rasterio.plot import show, show_hist
show(rast)

In [ ]:
show_hist(rast)

In [ ]:
rast.meta

In [ ]:
b5 = rast.read()

In [ ]:
print(type(b5))

In [ ]:
print(b5.shape)

### 1.2 Writing raster data
***
We will write (aka save) the data we opened and read previously to a new file with a different format (from GTiff to HFA).

In [ ]:
dst_meta = rast.meta

In [ ]:
dst_meta

In [ ]:
dst_meta.update(driver = 'HFA')

In [ ]:
dst_meta

In [ ]:
dst = r.open('b5.img', 'w', **dst_meta)

In [ ]:
dst.write(b5)

In [ ]:
dst.close()

In [ ]:
ls

## 2.0 Stacking image bands
***
**We will create a 3-band (RGB) multi-band stacked image from 3 individual tif files.**

In [ ]:
file_list = ['LC08_L1TP_045027_20210702_20210713_01_T1_B2.tiff',
             'LC08_L1TP_045027_20210702_20210713_01_T1_B3.tiff',
             'LC08_L1TP_045027_20210702_20210713_01_T1_B4.tiff']
file_list

In [ ]:
# Read metadata of first file
with r.open(file_list[0]) as src0:
    meta = src0.meta

In [ ]:
meta

In [ ]:
# Update meta to reflect the number of layers
meta.update(count = len(file_list))

In [ ]:
meta

In [ ]:
# Read each layer and write it to stack
with r.open('stack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with r.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [ ]:
ls

In [ ]:
stack = r.open("stack.tif")
print(type(stack))

In [ ]:
print(stack.count)
print(stack.height)
print(stack.width)

In [ ]:
show((stack, 1), cmap='Blues', title='Blue band')
show((stack, 1), cmap='Greens', title='Green band')
show((stack, 1), cmap='Reds', title='Red band')

In [ ]:
whole = stack.read()
show((whole - whole.min()) / (whole.max() - whole.min()))

In [ ]:
whole[1, 5000, 2000]

## 3.0 Clipping an image
***
### 3.1 First, we need a little aside about row/column and x/y coordinates
***

* The code below shows us the bounding box for the entire image

In [ ]:
stack.bounds

* This code below shows us the affine transform used to convert back and forth between row/column coordinates and x/y coordinates.
* See [here](https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html?highlight=affine) and [here](https://www.perrygeo.com/python-affine-transforms.html) for further information about affine transformations.

In [ ]:
stack.transform

In [ ]:
print(stack.width)
print(stack.height)

* Multiplying an images affine transform by a row/column coordinate pair yields an x/y coordinate pair.

In [ ]:
stack.transform * (3000, 3000)

* Alternatively, you can use rasterio in a slightly different way to do the same thing.

In [ ]:
r.transform.xy(stack.transform, 3000, 3000)

* And then you can do it back again using rasterio's r.transform.rowcol to convert x/y coordinates to row/col coordinates.
* And look, they match!

In [ ]:
r.transform.rowcol(stack.transform, 680400.0, 5285100.0)

### 3.1 Clipping using x/y image coordinates
***
**The x/y coordinates shown below are for an Area of Interest (AOI) in our image scene. It is the area we want to clip from the image.** Feel free to use a different AOI.

x1/y1 is the upper-left coordinate of the AOI. x2/y2 is the lower-right coordinate of the AOI.

In [ ]:
# x1 = 706041
# y1 = 5314251
# x2 = 720677
# y2 = 5300404

## Really small area
# x1 = 711882
# y1 = 5306769
# x2 = 712738
# y2 = 5305869

## Larger area
x1 = 725060
y1 = 5231848
x2 = 752214
y2 = 5216695

**The truth is, we don't really clip with x/y coordinates. We must clip with row/col coordinates. So...convert from x/y to row/col, as shown below.**

In [ ]:
ul = r.transform.rowcol(stack.transform, x1, y1)
lr = r.transform.rowcol(stack.transform, x2, y2)

In [ ]:
print(type(ul))
print(ul)
print(lr)

In [ ]:
print(f"The first element of the tuple is: {ul[0]} (this is the row) and the second element is: {ul[1]} (this is the column)")

In [ ]:
print(f"The upper left row/col coordinates are {ul}")
print(f"The lower right row/col coordinates are {lr}")

Previously we only ever used rasterio's read function without parameters. Now we will use a parameter...window. ```window``` is the bounding box of the area we want to clip and it has to be in row/column coordinates.

In [ ]:
subset = stack.read(window=((ul[0], lr[0]), (ul[1],lr[1])))

In [ ]:
subset[0, 0, 0]

In [ ]:
subset[0, 0, 1]

In [ ]:
subset.shape

In [ ]:
type(subset)

In [ ]:
subset

In [ ]:
show((subset - subset.min()) / (subset.max() - subset.min()))

#### Let's write this clipped image to a new file.
(Note that you can analyze the numpy array without ever writing it to disk. With a desktop software like ERDAS we are used to creating new files for every operation we run: clipping, stacking, processing, etc. You don't really need to do that when you are manipulating the code like this. This can be more efficient. When you need to, however, you can save to a new file.)


Below is the original meta dictionary. We must modify it to match our current numpy characteristics.

In [ ]:
meta

Everything is the same except for the number of rows and columns AND the transform--the coordinates of the transform are different.

In [ ]:
print(subset.shape)
print(subset.dtype)

In [ ]:
dst_meta = meta

In [ ]:
dst_meta.update(height=subset.shape[1])
dst_meta.update(width=subset.shape[2])

In [ ]:
from affine import Affine
t = Affine.translation(x1, y1) * Affine.scale(30, -30)
t

In [ ]:
dst_meta.update(transform = t)

In [ ]:
dst_meta

In [ ]:
dst = r.open('subset.tif', 'w', **dst_meta)
dst.write(subset)
dst.close()

In [ ]:
ls

## 4.0 Reprojecting an image
***

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

* First we determine the EPSG code for the new projection. Below we use 4326 which is WGS lon/lat.

In [ ]:
reproj_crs = 'EPSG:4326' # 'EPSG:3857'

In [ ]:
stack.bounds

* Second we use ```calculate_default_transform``` to determine the transform, height and width of the new projection.

In [ ]:
# dst_meta is the meta for the clipped image. Don't get confused and think it is for this reprojected image.
dst_transform, dst_width, dst_height = calculate_default_transform(dst_meta['crs'],
                                                                   reproj_crs, dst_meta['width'],
                                                                   dst_meta['height'],
                                                                   left=x1,
                                                                   bottom=y2,
                                                                   right=x2,
                                                                   top=y1)

In [ ]:
print(dst_transform)
print(dst_height)
print(dst_width)

In [ ]:
print("The x and y resolutions aren't really 0, they are " + str(dst_transform.a) + " and " + str(dst_transform.e) + ", respectively")

In [ ]:
reproj_meta = dst_meta.copy()

In [ ]:
reproj_meta.update({
    'crs': reproj_crs,
    'transform': dst_transform,
    'width': dst_width,
    'height': dst_height})

In [ ]:
reproj_meta

In [ ]:
dst = r.open('reproj.tif', 'w', **reproj_meta)
dst.write(subset)
dst.close()

In [ ]:
ls

Support for third party widgets will remain active for the duration of the session. To disable support: